In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import pickle
import logging

In [9]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("youcontrol_parser.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [10]:
def login_to_youcontrol(login, password):
    """Авторизація на сайті та збереження cookies."""
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--headless")

    driver = webdriver.Chrome(options=options)
    driver.get("https://youcontrol.com.ua/sign_in/")
    time.sleep(5)

    driver.find_element(By.NAME, "LoginForm[login]").send_keys(login)
    driver.find_element(By.NAME, "LoginForm[password]").send_keys(password)
    driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()

    time.sleep(5)
    cookies = driver.get_cookies()
    with open("cookies.pkl", "wb") as f:
        pickle.dump(cookies, f)

    logger.info("Авторизація виконана та cookies збережено.")
    return driver

In [11]:
def open_company_page(driver, edrpou_code):
    """Вводить ЄДРПОУ у пошук і відкриває сторінку компанії."""
    wait = WebDriverWait(driver, 15)
    try:
        search_box = wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "input[placeholder*='Введіть назву компанії']")
            )
        )
        search_box.clear()
        search_box.send_keys(edrpou_code)
        search_box.send_keys(Keys.ENTER)
        time.sleep(4)
        logger.info(f"Відкрито сторінку компанії з ЄДРПОУ {edrpou_code}")
    except Exception as e:
        logger.error(f"Не вдалося відкрити сторінку компанії {edrpou_code}: {e}")


In [12]:
def parse_company_youcontrol_page(html: str) -> dict:
    """Парсить сторінку компанії у структурований словник."""
    soup = BeautifulSoup(html, "lxml")
    result = {}

    def clean_text(text):
        return re.sub(r"\s+", " ", text.strip())

    # --- основна таблиця ---
    for table in soup.select("table.detail-view"):
        for tr in table.select("tr"):
            th, td = tr.find("th"), tr.find("td")
            if not th or not td:
                continue
            key = clean_text(th.get_text(" ", strip=True))
            val = clean_text(td.get_text(" ", strip=True))
            result[key] = val

    # --- EDRPOU ---
    edrpou = soup.find("h2", class_="seo-table-name case-icon short")
    if edrpou:
        match = re.search(r"\d+", edrpou.get_text())
        if match:
            result["EDRPOU_CODE"] = match.group()

    # --- Назва ---
    name = soup.find("h1") or soup.find("h2", class_="seo-table-name")
    if name:
        result["Company Name"] = clean_text(name.get_text())

    # --- Види діяльності ---
    activities_block = soup.select_one("div.localized-item.flex-activity")
    if activities_block:
        text = clean_text(activities_block.get_text(" ", strip=True))
        result["КВЕД (Основний)"] = text
    for idx, li in enumerate(soup.select("li.localized-item.localized-other")[:3], start=1):
        result[f"КВЕД_{idx} (Інші)"] = clean_text(li.get_text(" ", strip=True))

    # --- Бенефіціари ---
    beneficiary_block = soup.select("div#catalog-company-beneficiary div.seo-table-row")
    for idx, row in enumerate(beneficiary_block[:5], start=1):
        result[f"Beneficiary_{idx}"] = clean_text(row.get_text(" ", strip=True))

    return result

In [13]:
def run_parser(login, password, edrpou_csv):
    driver = login_to_youcontrol(login, password)
    df_codes = pd.read_csv(edrpou_csv, dtype=str)
    results = []

    for i, code in enumerate(df_codes.iloc[:, 0].dropna(), start=1):
        try:
            open_company_page(driver, code)
            time.sleep(3)
            html = driver.page_source
            data = parse_company_youcontrol_page(html)
            data["EDRPOU_INPUT"] = code
            results.append(data)
            logger.info(f"✅ [{i}] Спарсено компанію {code}")
            time.sleep(6)
        except Exception as e:
            logger.error(f"Помилка при парсингу {code}: {e}")
            continue

    df = pd.DataFrame(results)
    df.to_csv("youcontrol_companies.csv", index=False, encoding="utf-8-sig")
    logger.info(f"Збережено {len(df)} компаній у youcontrol_companies.csv")
    driver.quit()


In [14]:
if __name__ == "__main__":
    run_parser(
        login="nelkovvasilij@gmail.com",
        password="gEr32weIG&*",
        edrpou_csv=r"C:\OTP Draft\YouControl\test_dataset.csv"
    )


--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\Mykola\AppData\Local\Programs\Python\Python313\Lib\logging\__init__.py", line 1154, in emit
    stream.write(msg + self.terminator)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Mykola\AppData\Local\Programs\Python\Python313\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode characters in position 33-43: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Mykola\AppData\Roaming\Python\Python313\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Mykola\AppData\Roaming\Python\Python313\site-packages\traitlets\config\application.py", line 1075, in launch_instance
 

In [15]:
df_codes = pd.read_csv(r"C:\OTP Draft\YouControl\test_dataset.csv", dtype=str)
df_codes


,EDRPOU
0,45177892
1,45177908
2,45177913
3,45177929
4,45177934
5,45179423
6,45179444
7,45179453
8,45179465
9,45179486


In [16]:
df_results = pd.read_csv(r"C:\OTP Draft\YouControl\EDRPOU_approach_drafts\youcontrol_companies.csv")
df_results

,Повне найменування юридичної особи,Скорочена назва,Найменування іноземною мовою,Статус юридичної особи,Статус з ЄДР,Код ЄДРПОУ,Дата реєстрації,Уповноважені особи,Розмір статутного капіталу,Організаційно-правова форма,...,КВЕД_3 (Інші),EDRPOU_INPUT,Дані про перебування юридичної особи в процесі припинення,"Відомості про строк, визначений засновниками (учасниками) юридичної особи, судом або органом, що прийняв рішення про припинення юридичної особи, для заявлення кредиторами своїх вимог",Контакти з останнього тендеру (10.10.2025),Контактна особа:,"Головний бухгалтер або інша особа, уповноважена підписувати звітність",Веб сторінка:,Контакти з останнього тендеру (01.10.2025),Контакти з останнього тендеру (19.05.2025)
0,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ТАМЕН...","ТОВ ""ТАМЕН"" копіювати скопійовано",Повне найменування іноземною мовою LIMITED LIA...,Зареєстровано копіювати скопійовано,Зареєстровано копіювати скопійовано,45177892 копіювати скопійовано,06.09.2023 (2 роки 1 місяць) копіювати скопійо...,ТРИЗНО ЮЛІЯ СЕРГІЇВНА Відкрити в пошуку 9 — ди...,"31 181 400,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,Цей клас включає: технічне обслуговування та р...,45177892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ЮКСІС...","ТОВ ""ЮКСІС ТРЕЙД"" копіювати скопійовано",NaN,В стані припинення копіювати скопійовано,В стані припинення копіювати скопійовано,45177908 копіювати скопійовано,15.12.2023 (1 рік 9 місяців) копіювати скопійо...,ФРОЛОВ АНТОН ОЛЕГОВИЧ Відкрити в пошуку 9 — ке...,"50 000,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,Цей клас не включає: оптову торгівлю від власн...,45177908,25.06.2025 - в стані припинення 10022711000020...,25.08.2025,NaN,NaN,NaN,NaN,NaN,NaN
2,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ІНЖИН...","ТОВ ""ІК БУДІВЕЛЬНИЙ СВІТ"" копіювати скопійовано",Повне найменування іноземною мовою LIMITED LIA...,Зареєстровано копіювати скопійовано,Зареєстровано копіювати скопійовано,45177913 копіювати скопійовано,15.12.2023 (1 рік 9 місяців) копіювати скопійо...,СТОВБУН ОЛЕГ СЕРГІЙОВИЧ Відкрити в пошуку 5 — ...,"1 000,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,Цей клас включає: розбирання або злам будівель...,45177913,NaN,NaN,Контактна особа: Олег Стовбун E-mail: budiveln...,Олег Стовбун,NaN,NaN,NaN,NaN
3,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""АЛЬЯН...","ТОВ ""АЛЬЯНС АГРО МАР"" копіювати скопійовано",Повне найменування іноземною мовою LIMITED LIA...,Зареєстровано копіювати скопійовано,Зареєстровано копіювати скопійовано,45177929 копіювати скопійовано,15.12.2023 (1 рік 9 місяців) копіювати скопійо...,ПРИШЛЯК МАР'ЯНА СТЕПАНІВНА Відкрити в пошуку 5...,"200 000,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,"Цей клас включає: оптову торгівлю цукром, шоко...",45177929,NaN,NaN,NaN,NaN,ПРИШЛЯК МАР'ЯНА СТЕПАНІВНА Відкрити в пошуку 5...,NaN,NaN,NaN
4,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""Флафф...",NaN,Повне найменування іноземною мовою LIMITED LIA...,Зареєстровано копіювати скопійовано,Зареєстровано копіювати скопійовано,45177934 копіювати скопійовано,15.12.2023 (1 рік 9 місяців) копіювати скопійо...,ФЕЩУКОВ ГЛІБ ВОЛОДИМИРОВИЧ Відкрити в пошуку —...,"1 000,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,NaN,45177934,NaN,NaN,NaN,NaN,NaN,******:*/*/***********.****,NaN,NaN
5,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""СПЕЦГ...","ТОВ ""СПЕЦГУРТ"" копіювати скопійовано","Повне найменування іноземною мовою ""SPECGURT"" ...",Зареєстровано копіювати скопійовано,Зареєстровано копіювати скопійовано,45179423 копіювати скопійовано,20.07.2023 (2 роки 2 місяці) копіювати скопійо...,ЯМНЕНКО ВАДИМ ВАЛЕРІЙОВИЧ Відкрити в пошуку 2 ...,"5 000,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,Цей клас включає: оптову торгівлю побутовими м...,45179423,NaN,NaN,NaN,NaN,NaN,NaN,Адреса: **** м. Київ м. Київ вул.Бориспільська...,NaN
6,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""МІСТІ...","ТОВ ""МІСТІК ФЛЕЙМ"" копіюват

In [19]:
pd.read_csv(r"C:\OTP Draft\YouControl\EDRPOU_approach_drafts\youcontrol_companies.csv")

,Повне найменування юридичної особи,Скорочена назва,Найменування іноземною мовою,Статус юридичної особи,Статус з ЄДР,Код ЄДРПОУ,Дата реєстрації,Уповноважені особи,Розмір статутного капіталу,Організаційно-правова форма,...,КВЕД_3 (Інші),EDRPOU_INPUT,Дані про перебування юридичної особи в процесі припинення,"Відомості про строк, визначений засновниками (учасниками) юридичної особи, судом або органом, що прийняв рішення про припинення юридичної особи, для заявлення кредиторами своїх вимог",Контакти з останнього тендеру (10.10.2025),Контактна особа:,"Головний бухгалтер або інша особа, уповноважена підписувати звітність",Веб сторінка:,Контакти з останнього тендеру (01.10.2025),Контакти з останнього тендеру (19.05.2025)
0,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ТАМЕН...","ТОВ ""ТАМЕН"" копіювати скопійовано",Повне найменування іноземною мовою LIMITED LIA...,Зареєстровано копіювати скопійовано,Зареєстровано копіювати скопійовано,45177892 копіювати скопійовано,06.09.2023 (2 роки 1 місяць) копіювати скопійо...,ТРИЗНО ЮЛІЯ СЕРГІЇВНА Відкрити в пошуку 9 — ди...,"31 181 400,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,Цей клас включає: технічне обслуговування та р...,45177892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ЮКСІС...","ТОВ ""ЮКСІС ТРЕЙД"" копіювати скопійовано",NaN,В стані припинення копіювати скопійовано,В стані припинення копіювати скопійовано,45177908 копіювати скопійовано,15.12.2023 (1 рік 9 місяців) копіювати скопійо...,ФРОЛОВ АНТОН ОЛЕГОВИЧ Відкрити в пошуку 9 — ке...,"50 000,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,Цей клас не включає: оптову торгівлю від власн...,45177908,25.06.2025 - в стані припинення 10022711000020...,25.08.2025,NaN,NaN,NaN,NaN,NaN,NaN
2,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ІНЖИН...","ТОВ ""ІК БУДІВЕЛЬНИЙ СВІТ"" копіювати скопійовано",Повне найменування іноземною мовою LIMITED LIA...,Зареєстровано копіювати скопійовано,Зареєстровано копіювати скопійовано,45177913 копіювати скопійовано,15.12.2023 (1 рік 9 місяців) копіювати скопійо...,СТОВБУН ОЛЕГ СЕРГІЙОВИЧ Відкрити в пошуку 5 — ...,"1 000,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,Цей клас включає: розбирання або злам будівель...,45177913,NaN,NaN,Контактна особа: Олег Стовбун E-mail: budiveln...,Олег Стовбун,NaN,NaN,NaN,NaN
3,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""АЛЬЯН...","ТОВ ""АЛЬЯНС АГРО МАР"" копіювати скопійовано",Повне найменування іноземною мовою LIMITED LIA...,Зареєстровано копіювати скопійовано,Зареєстровано копіювати скопійовано,45177929 копіювати скопійовано,15.12.2023 (1 рік 9 місяців) копіювати скопійо...,ПРИШЛЯК МАР'ЯНА СТЕПАНІВНА Відкрити в пошуку 5...,"200 000,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,"Цей клас включає: оптову торгівлю цукром, шоко...",45177929,NaN,NaN,NaN,NaN,ПРИШЛЯК МАР'ЯНА СТЕПАНІВНА Відкрити в пошуку 5...,NaN,NaN,NaN
4,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""Флафф...",NaN,Повне найменування іноземною мовою LIMITED LIA...,Зареєстровано копіювати скопійовано,Зареєстровано копіювати скопійовано,45177934 копіювати скопійовано,15.12.2023 (1 рік 9 місяців) копіювати скопійо...,ФЕЩУКОВ ГЛІБ ВОЛОДИМИРОВИЧ Відкрити в пошуку —...,"1 000,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,NaN,45177934,NaN,NaN,NaN,NaN,NaN,******:*/*/***********.****,NaN,NaN
5,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""СПЕЦГ...","ТОВ ""СПЕЦГУРТ"" копіювати скопійовано","Повне найменування іноземною мовою ""SPECGURT"" ...",Зареєстровано копіювати скопійовано,Зареєстровано копіювати скопійовано,45179423 копіювати скопійовано,20.07.2023 (2 роки 2 місяці) копіювати скопійо...,ЯМНЕНКО ВАДИМ ВАЛЕРІЙОВИЧ Відкрити в пошуку 2 ...,"5 000,00 грн",ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ копіюв...,...,Цей клас включає: оптову торгівлю побутовими м...,45179423,NaN,NaN,NaN,NaN,NaN,NaN,Адреса: **** м. Київ м. Київ вул.Бориспільська...,NaN
6,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""МІСТІ...","ТОВ ""МІСТІК ФЛЕЙМ"" копіюват